# **Air Pollution in Seoul**
---
[<img src="https://i.guim.co.uk/img/media/1d1abc561aad84be09d26b81d6959cb34c9be236/0_10_5000_3003/master/5000.jpg?width=1200&quality=85&auto=format&fit=max&s=cc0522633035a7c9ffd7081935b3a462" width="500"/>](image.png)

Es de conocimiento común que la contaminación del aire puede causar varios problemas en el medio ambiente y en nuestra salud. La foto de arriba fue tomada el 11 de diciembre de 2019 y muestra cómo puede impactar severamente los paisajes de Seúl. En esta ocasión, un smog de polvo ultrafino, proveniente desde China, duró dos días e hizo que el gobierno local dictara medidas de emergencia para la reducción de emisiones. Según The Korea Times, el Centro de Pronóstico de la Calidad del Aire, con el Ministerio de Medio Ambiente, informó que el 11 de diciembre a las 10 p.m la concentración de partículas PM2.5 era de aproximadamente 118 microgramos por metro cúbico en Seúl.

# **Objetivo Investigacion del dataset**
El proposito de trabajo de este dataset es para lograr dar una prediccion de como sera la contanimacion dentro una ciudad, en este caso el conjunto de datos nos proporciona la informacion de medicion de la contaminacion del aire en Seul, Corea del Sur.

Estos datos proporcionan valores promedio para seis contaminantes $(SO_2, NO_2, CO, O3, PM_{10}, PM_{2.5}).$
- Los datos se midieron cada hora entre 2017 y 2019.
- Se midieron los datos de 25 distritos de Seúl.


- **Measurement** date = Hora y fecha de cuando se midio
- **Station code** = Codigo de Estacion
- **Address** = Direccion donde se obtuvieron los datos
- **Latitude** = Latitud
- **Longitude** = Longitud 
- **$SO_{2}$** = El dióxido de azufre, u óxido de azufre 
- **$NO_2$** = Dióxido de nitrógeno
- **$CO$** = Monóxido de carbono
- **$O3$** = Ozono
- **$PM_{10}$** = Pequeñas partículas sólidas o líquidas de polvo, cenizas, hollín, partículas metálicas, cemento o polen, dispersas en la atmósfera
- **$PM_{2.5}$** = Pequeñas partículas que incluye sustancias químicas orgánicas, polvo, hollín y metales. 
- **class** = Interpretacion si la contaminacion es:
  - Good : Es normal
  - Bad: Es mala 

In [218]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

## Lectura e Importacion del Conjunto de Datos

In [230]:
df = pd.read_csv('https://gitlab.com/sergiopaucara/datos-air-pollution-in-seoul/-/raw/main/Measurement_summary.csv')
df = df[:-1]
df.head()

,Unnamed: 0,Measurement date,Station code,Address,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5,class
0,0,2017-01-01 00:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005007,0.004,0.059,0.002,1.2,73.0,57.0,Good
1,1,2017-01-01 01:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005007,0.004,0.058,0.002,1.2,71.0,59.0,Good
2,2,2017-01-01 02:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005007,0.004,0.056,0.002,1.2,70.0,59.0,Good
3,3,2017-01-01 03:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005007,0.004,0.056,0.002,1.2,70.0,58.0,Good
4,4,2017-01-01 04:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005007,0.003,0.051,0.002,1.2,69.0,61.0,Good


In [244]:
X = df.iloc[:, [2, 4, 5, 6, 7, 8, 9, 10, 11]].values
y = df.iloc[:, -1].values

## **Preprocesamiento de datos**

### **Prepocesamiento 1 - Ejemplo**

Transformador de imputación para completar valores perdidos.

In [245]:
from sklearn.impute import SimpleImputer
simple = SimpleImputer()
simple.fit(X)
X_1 = simple.transform(X)
X_1

array([[101.       ,  37.5720164, 127.0050075, ...,   1.2      ,
         73.       ,  57.       ],
       [101.       ,  37.5720164, 127.0050075, ...,   1.2      ,
         71.       ,  59.       ],
       [101.       ,  37.5720164, 127.0050075, ...,   1.2      ,
         70.       ,  59.       ],
       ...,
       [125.       ,  37.5449625, 127.1367917, ...,   0.4      ,
         25.       ,  19.       ],
       [125.       ,  37.5449625, 127.1367917, ...,   0.4      ,
         24.       ,  17.       ],
       [125.       ,  37.5449625, 127.1367917, ...,   0.5      ,
         25.       ,  18.       ]])

### **Prepocesamiento 2 - Ejemplo**

Transforme las características escalando cada característica a un rango determinado.

Este estimador escala y traduce cada característica individualmente de modo que esté en el rango dado en el conjunto de entrenamiento, p. Ej. entre cero y uno.

In [246]:
from sklearn.preprocessing import MinMaxScaler
escala=MinMaxScaler()
escala.fit(X)
X_2 =escala.transform(X)
X_2

array([[0.        , 0.57969677, 0.56310927, ..., 0.03026135, 0.02063005,
        0.00926962],
       [0.        , 0.57969677, 0.56310927, ..., 0.03026135, 0.02007248,
        0.00958926],
       [0.        , 0.57969677, 0.56310927, ..., 0.03026135, 0.0197937 ,
        0.00958926],
       ...,
       [1.        , 0.44863272, 1.        , ..., 0.01925722, 0.0072484 ,
        0.00319642],
       [1.        , 0.44863272, 1.        , ..., 0.01925722, 0.00696961,
        0.00287678],
       [1.        , 0.44863272, 1.        , ..., 0.02063274, 0.0072484 ,
        0.0030366 ]])

### **Prepocesamiento 3 - Valido**

Estandarice las características eliminando la media y escalando a la varianza de la unidad.

La puntuación estándar de una muestra $x$ se calcula como:

$ z = (x - u) / s $

Donde $u$ es la media de las muestras de entrenamiento o cero si `with_mean = False`, y $s$ es la desviación estándar de las muestras de entrenamiento o uno si `with_std = False`.

In [247]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
X

array([[-1.66408315,  0.34788437,  0.19885863, ...,  1.70434583,
         0.4117658 ,  0.7191414 ],
       [-1.66408315,  0.34788437,  0.19885863, ...,  1.70434583,
         0.38365116,  0.76467397],
       [-1.66408315,  0.34788437,  0.19885863, ...,  1.70434583,
         0.36959385,  0.76467397],
       ...,
       [ 1.66402704, -0.1599516 ,  1.87145932, ..., -0.26941053,
        -0.26298534, -0.14597758],
       [ 1.66402704, -0.1599516 ,  1.87145932, ..., -0.26941053,
        -0.27704265, -0.19151016],
       [ 1.66402704, -0.1599516 ,  1.87145932, ..., -0.02269099,
        -0.26298534, -0.16874387]])

### **Prepocesamiento 4 - Valido**
Codifique las etiquetas de destino con un valor entre 0 y n_classes-1.

Este transformador debe usarse para codificar valores objetivo, es decir, $y$, y no la entrada $X$.

In [248]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
y[:] = le.fit_transform(y[:])
y = y.astype('int')

# **Clasificador**
El modelo de regresión logística general tiene múltiples variables explicativas que pueden ser cuantitativas, categóricas o ambas. Para las variables explicativas de $ p $, el modelo para las probabilidades logarítmicas es

$$ logit [P (Y = 1)] = α + β_1x_1 + β_2x_2 + ··· + β_px_p $$

El parámetro $ β_j $ se refiere al efecto de $ x_j $ en las probabilidades de registro de que $ Y = 1 $, ajustando los de los otros $x's$. Por ejemplo, $ \ exp (β_1) $ es el efecto multiplicativo sobre las probabilidades de un aumento de 1 unidad en $ x_1 $, a un valor fijo para $ β_2x_2 + \cdots + β_px_p $, como cuando podemos mantener constante $ x_2, ..., x_p $.

## **Primera Ejecucion**



In [249]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [250]:
# Crear un objeto de regresión logística, realizar una regresión logística
log_reg = LogisticRegression(solver = 'lbfgs', max_iter=1000)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [251]:
# Predecir la respuesta para el conjunto de datos de prueba
y_pred = log_reg.predict(X_test)

In [252]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result)
print("Exactitud del modelo:", accuracy_score(y_test, y_pred))

Confusion Matrix:
[[  6291   3293]
 [  1290 118628]]
Exactitud del modelo: 0.9646105851647079


In [262]:
from sklearn.model_selection import StratifiedShuffleSplit
cv = StratifiedShuffleSplit(n_splits = 100, test_size=0.2, random_state=0)
iter = cv.split(X, y)
split = []
for index, (train_index, test_index) in enumerate(iter):
  print('SPLIT: ', index + 1)
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  log_reg = LogisticRegression(solver = 'lbfgs', max_iter=1000)
  log_reg.fit(X_train, y_train)
  y_pred = log_reg.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  split.append(accuracy)
  print('Exactitud del Modelo', accuracy)

SPLIT:  1
TRAIN: [112686 198008  32508 ... 420908 427172 413699] TEST: [623950 594476 592772 ... 516776 410002  81049]
Exactitud del Modelo 0.9651356735803308
SPLIT:  2
TRAIN: [355405  20765 501751 ... 578377 555229 151640] TEST: [309248 135555 558425 ... 534456  49367 239487]
Exactitud del Modelo 0.9637920649874133
SPLIT:  3
TRAIN: [246161 414516 528472 ... 582566 567516 245642] TEST: [376161 526223 472326 ... 150167 173992 372735]
Exactitud del Modelo 0.964811354264799
SPLIT:  4
TRAIN: [ 38546 274323 381864 ... 299471  77379  45302] TEST: [175036 507293 425540 ... 540222 353467 184156]
Exactitud del Modelo 0.9646800821608933
SPLIT:  5
TRAIN: [115220 143626 634247 ... 229104 133437 506096] TEST: [624214 578946 398971 ... 190585 134388 413425]
Exactitud del Modelo 0.9643248752915012
SPLIT:  6
TRAIN: [443149 178789 626826 ... 329673 112308 312380] TEST: [186422  26343 403708 ...   6383 364244  88851]
Exactitud del Modelo 0.9648499637071242
SPLIT:  7
TRAIN: [391626   2412 389556 ... 2805

In [263]:
split = np.array(split)
print('Primera ejecucion 80(train) - 20(test)')
print('Mediana de la Confiabilidad: ', np.median(split))

Primera ejecucion 80(train) - 20(test)
Mediana de la Confiabilidad:  0.9645565319454525


In [265]:
cv = StratifiedShuffleSplit(n_splits = 100, test_size=0.5, random_state=0)
iter = cv.split(X, y)
split_2 = []
for index, (train_index, test_index) in enumerate(iter):
  print('SPLIT: ', index + 1)
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  log_reg = LogisticRegression(solver = 'lbfgs', max_iter=1000)
  log_reg.fit(X_train, y_train)
  y_pred = log_reg.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  split_2.append(accuracy)
  print('Exactitud del Modelo', accuracy)


SPLIT:  1
TRAIN: [515898 263945 402627 ... 408913 139508  37600] TEST: [170239 631015 291453 ...  76455 288151 589059]
Exactitud del Modelo 0.9648777625055984
SPLIT:  2
TRAIN: [404763 174232 624126 ... 267864 270561  58844] TEST: [  6027 142932  74813 ... 356535 158453 185879]
Exactitud del Modelo 0.964627573319331
SPLIT:  3
TRAIN: [242487  21856 364320 ... 603612 626892 421699] TEST: [401763 316095 572419 ... 622212 447926 541270]
Exactitud del Modelo 0.964979691433337
SPLIT:  4
TRAIN: [342340  95864 445978 ... 623319 313287 183261] TEST: [445709 186780 290310 ... 403172 380401  73855]
Exactitud del Modelo 0.9643619403561335
SPLIT:  5
TRAIN: [250791  25073 558351 ... 180686 586396  17415] TEST: [  2411 379535 185373 ... 417283 551633 300691]
Exactitud del Modelo 0.9648623187286682
SPLIT:  6
TRAIN: [313203  91195 187763 ... 538819 398612 411591] TEST: [413673 581302 126284 ...  24438 407171 258140]
Exactitud del Modelo 0.9651094191595496
SPLIT:  7
TRAIN: [378704 623658 109284 ... 59829

In [266]:
split_2 = np.array(split_2)
print('Segunda ejecucion 50(train) - 50(test)')
print('Mediana de la Confiabilidad: ', np.median(split_2))

Segunda ejecucion 50(train) - 50(test)
Mediana de la Confiabilidad:  0.9645040231038903


# **PCA - Principal Component Analysis**

PCA siempre se puede utilizar para simplificar los datos con grandes dimensiones (mayores de 2) en datos bidimensionales eliminando las características menos influntiales de los datos. Sin embargo, debemos saber que la eliminación de datos hace que la variable independiente sea menos interpretable. Antes de comenzar a tratar con el PCA, primero debemos aprender cómo el PCA utiliza los vectores propios para obtener una matriz de covarianza de diagonalización.

## **Autovectores**

Los autovectores y autovalores son las principales herramientas utilizadas por PCA para obtener una matriz de covarianza de diagnóstico. El autovector es un vector cuya dirección no se verá afectada por la transformación lineal, por lo tanto, los autovectores representan la dirección de mayor varianza de los datos, mientras que el autovalor decide la magnitud de esta varianza en esas direcciones.

Aquí usamos una matriz simple (2x2) $ A $ para explicarlo.
$$
A = \begin {bmatrix}
1 & 4 \\
3 & 2
\end {bmatrix}
$$

En general, el vector propio $v$ de una matriz $A$ es el vector donde se cumple lo siguiente:

$$
Av = \lambda v
$$

para el cual $\lambda$ representa el autovalor tal que la transformación lineal en $v$ puede definirse mediante $\lambda$

Además, podemos resolver la ecuación de la siguiente manera:

$$
Av - \lambda v = 0 \\
v(A-\lambda I) = 0
$$
Mientras que $I$ es la matriz identidad de A

$$
I = A^TA = AA^T
$$

En este caso, si $v$ es un vector sin cero entonces $Det(A - \lambda I) = 0$, ya que no puede ser invertible, y podemos resolver $v$ para $A$ depende de esta relación.

$$
I = \begin{bmatrix} 
1 & 0 \\
0 & 1 
\end{bmatrix} \\
$$

$$
(A - \lambda I) = \begin{bmatrix}
1-\lambda & 4 \\
3 & 2 - \lambda 
\end{bmatrix} \\
$$

Para resolver el $\lambda$ podemos usar la función resolver en sympy o calculando.

En este caso, $\lambda_1 = -2 $ y $ \lambda_2 = 5 $, y podemos calcular los vectores propios en dos casos.

Por $ \lambda_1 = -2 $

Con base en la matriz, podemos inferir que el vector propio puede ser
$$
v_1 = \begin {bmatrix}
-4 \\
3 \end {bmatrix}
$$

Por $ \lambda = 5 $

Con base en la matriz, podemos inferir que el vector propio puede ser
$$
v_2 = \begin {bmatrix}
1 \\
1 \end {bmatrix}
$$
Con todo, la matriz de covarianza $ A'$ ahora puede ser:
$$
A' = v * A \\
$$

De tal manera que podamos obtener la matriz $V$
$$
V = \begin {bmatrix}
-4 & 1 \\
3 & 1
\end {bmatrix}
$$
donde $ A' = V ^ {- 1} A V $ 
# Ejecutando PCA


In [333]:
cov_mat = np.dot(X.T, X)
cov_mat

array([[ 6.47510000e+05, -3.17134965e+05,  3.04718613e+04,
         1.78240818e+04,  1.39381385e+04,  1.22972438e+04,
        -2.25013534e+04,  1.95184392e+04,  1.12813560e+04],
       [-3.17134965e+05,  6.47510000e+05,  1.99162748e+05,
        -4.40090867e+03, -1.35728720e+04,  2.41555797e+02,
         3.65483233e+04, -1.41639443e+04, -1.49420706e+04],
       [ 3.04718613e+04,  1.99162748e+05,  6.47510000e+05,
         9.84785032e+03,  5.55641763e+03,  4.79069353e+03,
         2.29083675e+04, -1.47346880e+04, -1.22591124e+04],
       [ 1.78240818e+04, -4.40090867e+03,  9.84785032e+03,
         6.47510000e+05,  4.61300409e+05,  5.21602189e+05,
         1.97440402e+05,  3.14516841e+04,  3.07768689e+04],
       [ 1.39381385e+04, -1.35728720e+04,  5.55641763e+03,
         4.61300409e+05,  6.47510000e+05,  5.08816925e+05,
         1.59122795e+05,  3.59576893e+04,  3.74545357e+04],
       [ 1.22972438e+04,  2.41555797e+02,  4.79069353e+03,
         5.21602189e+05,  5.08816925e+05,  6.475100

In [334]:
autovalores, autovectores = np.linalg.eig(cov_mat)
print(autovalores)
print(autovectores)

[1727870.83882243 1016466.95409743  107188.72579891  259164.27252604
  185141.75603897  845714.30088854  674470.3096866   519889.80291888
  491683.0392222 ]
[[-0.02173509 -0.59545712 -0.00404949  0.59602315  0.04662498 -0.04094427
   0.53235712  0.04129027 -0.02718651]
 [ 0.00635899  0.71310549 -0.02703131  0.69058172  0.05252777  0.03882337
   0.03118996 -0.08799049  0.02871867]
 [-0.01321286  0.34628096  0.01115557 -0.40156531 -0.0235633  -0.0182809
   0.84393135 -0.05765174  0.04585307]
 [-0.5552525   0.00459065 -0.51165811  0.03375465 -0.64649305 -0.10243596
  -0.00919118 -0.01388406  0.00406858]
 [-0.54468574 -0.0073891  -0.33366724 -0.0650103   0.74901775 -0.11207502
  -0.02366649 -0.0963212   0.0653942 ]
 [-0.55820865  0.0031062   0.78345968  0.02733587 -0.10871726 -0.16434567
  -0.02994288 -0.15826454  0.09520479]
 [-0.26152009  0.09070118  0.11031143 -0.0029807   0.05499375  0.40123995
   0.04085239  0.7357603  -0.45185517]
 [-0.08303581 -0.07237009 -0.00364312 -0.02054758 -0.

In [357]:
from random import shuffle
components = 2
v = [i for i in range(9)]
shuffle(v)
pca = np.delete(autovectores, obj = v[:9 - components], axis = 1)

var_acc = (np.sum(autovalores) - autovalores[2]) / (np.sum(autovalores))
print(var_acc)

0.9816066803260156


In [351]:
pca

array([[-0.59545712,  0.04129027],
       [ 0.71310549, -0.08799049],
       [ 0.34628096, -0.05765174],
       [ 0.00459065, -0.01388406],
       [-0.0073891 , -0.0963212 ],
       [ 0.0031062 , -0.15826454],
       [ 0.09070118,  0.7357603 ],
       [-0.07237009, -0.63448148],
       [-0.0587488 ,  0.09381306]])

In [352]:
data_pca = np.dot(X, pca)
print(len(data_pca[0]))
data_pca

2


array([[ 1.38786405,  0.94334064],
       [ 1.38728789,  0.96628687],
       [ 1.38843355,  0.9768789 ],
       ...,
       [-0.45366914, -0.06834017],
       [-0.44984849, -0.06201969],
       [-0.43126047,  0.11603394]])

In [353]:
X_train, X_test, y_train, y_test = train_test_split(data_pca, y, test_size=0.2)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
result = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result)
print("Exactitud del modelo:", accuracy_score(y_test, y_pred))

Confusion Matrix:
[[     3   9571]
 [     0 119928]]
Exactitud del modelo: 0.9260938055010733


# **PCA con Python**

In [144]:
import mglearn
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline 

### **PCA - 2 componentes**

In [296]:
pca=PCA(n_components=2)
pca.fit(X)
transformada=pca.transform(X)

X_train, X_test, y_train, y_test = train_test_split(transformada, y, test_size=0.2)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
result = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result)
print("Exactitud del modelo:", accuracy_score(y_test, y_pred))


Confusion Matrix:
[[  4737   4856]
 [  1291 118618]]
Exactitud del modelo: 0.9525335516053806


### **PCA - 3 componentes**

In [286]:
pca=PCA(n_components=3)
pca.fit(X)
transformada=pca.transform(X)

X_train, X_test, y_train, y_test = train_test_split(transformada, y, test_size=0.2)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
result = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result)
print("Exactitud del modelo:", accuracy_score(y_test, y_pred))

Confusion Matrix:
[[  5021   4575]
 [  1289 118617]]
Exactitud del modelo: 0.9547188460409878


### **PCA - 5 componentes**

In [287]:
pca=PCA(n_components=5)
pca.fit(X)
transformada=pca.transform(X)

X_train, X_test, y_train, y_test = train_test_split(transformada, y, test_size=0.2)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
result = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result)
print("Exactitud del modelo:", accuracy_score(y_test, y_pred))

Confusion Matrix:
[[  5233   4360]
 [  1381 118528]]
Exactitud del modelo: 0.9556686383221881


### **PCA - 7 componentes**

In [288]:
pca=PCA(n_components=7)
pca.fit(X)
transformada=pca.transform(X)

X_train, X_test, y_train, y_test = train_test_split(transformada, y, test_size=0.2)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
result = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result)
print("Exactitud del modelo:", accuracy_score(y_test, y_pred))

Confusion Matrix:
[[  5364   4346]
 [  1372 118420]]
Exactitud del modelo: 0.955846241756884


### **PCA - 8 componentes**

In [289]:
pca=PCA(n_components=8)
pca.fit(X)
transformada=pca.transform(X)

X_train, X_test, y_train, y_test = train_test_split(transformada, y, test_size=0.2)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
result = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result)
print("Exactitud del modelo:", accuracy_score(y_test, y_pred))

Confusion Matrix:
[[  6354   3407]
 [  1362 118379]]
Exactitud del modelo: 0.9631743139102099
